# Sign Language Classification Problem

The dataset format is patterned to match closely with the classic MNIST. Each training and test case represents a label (0-25) as a one-to-one map for each alphabetic letter A-Z. The training data (27,455 cases) and test data (7172 cases) are approximately half the size of the standard MNIST but otherwise similar with a header row of label, 

pixel1,pixel2….pixel784 which represent a single 28x28 pixel image with grayscale values between 0-255.


In [ ]:
import os
from IPython.display import Image
Image(filename="../input/sign-language-mnist/amer_sign2.png", width= 800, height=500)

<h1> 0. List the Directory </h1>

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

<h1> 1. Importing important Libraries </h1>

In [ ]:
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

We need to read the csv train and test inputs. Since we are training these as images, so we need to convert them to images and extract labels from it.

The 1st column of the csv has the label information and the rest are the image pixels.
We'll return the images and labels as numpy array.

<h1> 2. Reading the dataset </h1>

In [ ]:
def get_data(filename):
    with open(filename) as training_file:
        training_reader = csv.reader(training_file, delimiter=',')
        image = []
        labels = []
        line_count = 0
        for row in training_reader:
            if line_count == 0:
                line_count +=1
            else:
                labels.append(row[0])
                temp_image = row[1:785]
                image_data_as_array = np.array_split(temp_image, 28)
                image.append(image_data_as_array)
                line_count += 1
        images = np.array(image).astype('float')
        labels = np.array(labels).astype('float')
        print(f'Processed {line_count} lines.')

    return images, labels


training_images, training_labels = get_data("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
testing_images, testing_labels = get_data("../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")

print("Total Training images", training_images.shape)
print("Total Training labels",training_labels.shape)
print("Total Testing images",testing_images.shape)
print("Total Testing labels",testing_labels.shape)

<h1>3. EDA and Data Visualization </h1>

In [ ]:
alphabets = 'abcdefghijklmnopqrstuvwxyz'
mapping_letter = {}

for i,l in enumerate(alphabets):
    mapping_letter[l] = i
mapping_letter = {v:k for k,v in mapping_letter.items()}

In [ ]:
# Display some pictures of the dataset
fig, axes = plt.subplots(nrows=4, ncols=6, figsize=(8, 8),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    img = training_images[i].reshape(28,28)
    ax.imshow(img, cmap = 'gray')
    title = mapping_letter[training_labels[i]]
    ax.set_title(title, fontsize = 15)
plt.tight_layout(pad=0.5)
plt.show()

In [ ]:
# Display the distribution of each letter

vc = pd.Series(training_labels).value_counts()
plt.figure(figsize=(20,5))
sns.barplot(x = sorted(vc.index), y = vc, palette = "rocket")
plt.title("Number of pictures of each category", fontsize = 15)
plt.xticks(fontsize = 15)
plt.show()

As you can see that there are 25 categories present in the labels, On careful observation we find that **Z** is not present in the dataset.

Now we need to add another dimension in our images so that we can process it for the **ImageDataGenerator** and do the **Image Augmentation**
Read more [here](https://keras.io/api/preprocessing/image/)

<h1> 4. Data Augmentation </h1> 

In [ ]:
training_images = np.expand_dims(training_images, axis = 3)
testing_images = np.expand_dims(testing_images, axis = 3)

print(training_images.shape)
print(testing_images.shape)

In [ ]:
# Create an ImageDataGenerator and do Image Augmentation

train_datagen = ImageDataGenerator(rescale = 1.0/255.0,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1,
                                   zoom_range=0.1,
                                   shear_range=0.1,
                                   rotation_range=10,
                                   fill_mode='nearest',
                                   horizontal_flip=True)

#Image Augmentation is not done on the testing data

validation_datagen = ImageDataGenerator(rescale=1.0/255)

train_datagenerator = train_datagen.flow(training_images,
                                         training_labels,
                                         batch_size = 32)

validation_datagenerator = validation_datagen.flow(testing_images,
                                                   testing_labels, 
                                                   batch_size=32)


Now lets define a callback for avoiding the excess training and stopping the training based on the predefined condition, in our case, we want training to stop once the **accuracy** is reached above **99%**.

<h1>5.  Define a Callback </h1>

In [ ]:
# Define a Callback class that stops training once accuracy reaches 99.8%

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.998):
      print("\nReached 99.8% accuracy so cancelling training!")
      self.model.stop_training = True

<h1> 6. Defining the model </h1>

In [ ]:
# Define the model

model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape = (28,28,1)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(1024, activation = 'relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(512, activation = 'relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(25, activation = 'softmax')])

In [ ]:
model.summary()

I have used 3 Conv2D and 3 MaxPooling2D and the dropout of 0.2

In [ ]:
# Compiling the Model. 
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

<h1>7. Learning Rate modification </h1>

In [ ]:
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience = 2, 
                                            verbose=1,factor=0.25, 
                                            min_lr=0.0001)

<h1> 8. Training Model </h1>

In [ ]:
# Train the Model
callbacks = myCallback()
history = model.fit(train_datagenerator,
                    validation_data = validation_datagenerator,
                    steps_per_epoch = len(training_labels)//32,
                    epochs = 100,
                    validation_steps = len(testing_labels)//32,
                    callbacks = [callbacks, learning_rate_reduction])

<h1> 9. Plotiing the losses </h1>

In [ ]:
# Plot the chart for accuracy and loss on both training and validation

import matplotlib.pyplot as plt
fig.set_size_inches(16,9)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.evaluate(testing_images, testing_labels, verbose=0)

# model.save('sign_language.h5')

<h1> 10. Visualise the model </h1>

In [ ]:
tf.keras.utils.plot_model(model,
                          to_file="model.png",
                          show_shapes=True,
                          show_dtype=False,
                          show_layer_names=True,
                          rankdir="TB",                          
                          expand_nested=True,
                          dpi=96)

<h1>11. Evaluating Model </h1>

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
# Predict the label of the test_images
pred = model.predict(testing_images)
pred = np.argmax(pred,axis=1)

# Get the accuracy score
acc = accuracy_score(testing_labels,pred)

# Display the results
print(f'## {acc*100:.2f}% accuracy on the test set')

In [ ]:
# Map the numbers into letters
y_test_letters = [mapping_letter[x] for x in testing_labels]
pred_letters = [mapping_letter[x] for x in pred]

print(classification_report(y_test_letters, pred_letters))

In [ ]:
# Display a confusion matrix
cf_matrix = confusion_matrix(y_test_letters, pred_letters, normalize='true')
plt.figure(figsize = (20,15))
sns.heatmap(cf_matrix, annot=True, xticklabels = sorted(set(y_test_letters)), yticklabels = sorted(set(y_test_letters)),cbar=False)
plt.title('Normalized Confusion Matrix\n', fontsize = 23)
plt.xlabel("Predicted Classes",fontsize=15)
plt.ylabel("True Classes",fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15,rotation=0)
plt.show()

<h1>12. Outputs sample</h1>

In [ ]:
correct = np.nonzero(pred == testing_labels)[0]
plt.figure(figsize=(6, 6))
i = 0
for c in correct[:9]:
    plt.subplot(3,3,i+1)
    plt.imshow(testing_images[c].reshape(28,28), cmap="gray", interpolation='none')
    plt.title("Predicted:{}, Actual:{}".format(pred_letters[c], y_test_letters[c]))
    plt.tight_layout()
    i += 1